Optimal control of a Schrödinger type equation with an operator splitting + gradient descent methods

In [ ]:
from math import *
from numpy import *
from scipy import linalg
T = 10
L = 20 # number of step
t = linspace(0,T,L)
# print(t)
dt = t[2]-t[1]
#print("dt=" +str(dt))
H0 = array([[1, 0], [0, 2]])
H1 = array([[0, 1], [1, 0]])
y0 = [1, 0]
y0 = transpose(y0)
ycible = [0, 1]
ycible = transpose(ycible)
c = ones((1,L-1))
alpha = 10
maxiter = 30
#shape(c)

Calculons y

In [ ]:
def splitting (H0, H1, dt, L, c, y0):
    y = zeros((2,L))
    y[:,0] = y0

    for i in range(1,L):
        y[:,i] = linalg.expm((-1j)*H0*dt)@linalg.expm((-1j)*H1*c[:,i-1]*dt)@y[:,i-1]
    return y

In [ ]:
y = splitting(H0, H1, dt, L, c, y0)
#print(shape(y))

Calculons J

In [ ]:
def fonctionnelle (H0, H1, dt, L, c, ycible, y):
    alpha = 10
    yL=y[:,L-1]
    c = c**2 # on eleve chaque terme de c au carree
    c=sum(c) # somme des termes de c
    return 1/2*linalg.norm(yL-ycible)**2 + alpha * dt * c

In [ ]:
J = fonctionnelle(H0, H1, dt, L, c, ycible, y)
print(J)

Calculons p

In [ ]:
def calculP (H0, H1, dt, L, c, ycible, y):
    p = zeros((2,L))
    yL = y[:,L-1]
#     print(yL-ycible)
#     print(p[:,L-1])
    p[:,L-1] = yL-ycible # p(T) de l'equation (3), yL=y(T)
    for i in range(L-2,-1,-1): # backward range(start,end+1,pas)
        p[:,i] = linalg.expm((-1j)*H1*c[:,i]*dt)@linalg.expm((-1j)*H0*dt)@p[:,i+1]
    return p

# for i in range(20,-1,-1):
#     print(i, end=" ")

In [ ]:
p = calculP (H0, H1, dt, L, c, ycible, y)
#print(shape(p))

Calculons gradJ

In [ ]:
def gradJ (H0, H1, dt, L, c, ycible, y):
    alpha = 10
    p = calculP(H0, H1, dt, L, c, ycible, y)
    p = transpose(p)
    grad = 2*alpha*dt*c
    for i in range(0,L-1):
        grad[:,i] = grad[:,i] + real(p[i+1:i+2:,]@linalg.expm((-1j)*H0*dt)@((-1j)*H1*dt)@linalg.expm((-1j)*H1*c[:,i]*dt)@y[:,i])
    return grad

In [ ]:
grad = gradJ (H0, H1, dt, L, c, ycible, y)
#print(shape(grad))

Debug gradient: On verifie que le taux de variation - gradJ tend bien vers 0

In [ ]:
def debuggrad (H0, H1, dt, L, c, ycible, y):
    c = random.rand(1,L-1)
    dc = random.rand(1,L-1)
    grad = gradJ (H0, H1, dt, L, c, ycible, y)
    for i in range(1,7):
        epsi = 10**(-i)
        Jp = fonctionnelle (H0, H1, dt, L, c+epsi*dc, ycible, y)
        Jm = fonctionnelle (H0, H1, dt, L, c-epsi*dc, ycible, y)
        conv = (Jp-Jm)/(2*epsi)-grad@(transpose(dc))
        print(conv)
    

In [ ]:
debuggrad (H0, H1, dt, L, c, ycible, y)

Calculons enfin c

In [ ]:
def controle(H0, H1, dt, L, c, ycible, y):
    rho = 0.1
    for i in range(1,maxiter):
        J = fonctionnelle(H0, H1, dt, L, c, ycible, y)
        grad = gradJ(H0, H1, dt, L, c, ycible, y)
        c = c - rho*grad
        print(linalg.norm(grad))

In [ ]:
c = controle(H0, H1, dt, L, c, ycible, y)